In [1]:
import functools
import itertools
import pdb
import os
import math
import random

import hydra
from hydra.experimental import compose, initialize
import scipy.stats
import numpy as np
import skimage.io
from omegaconf import OmegaConf
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec

import precog.utils.log_util as logu
import precog.utils.tfutil as tfutil
import precog.interface as interface
import precog.plotting.plot as plot
import precog.plotting as plotting
        
def seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
seed(1234)

MODEL_DIRECTORY = "/home/fireofearth/code/data/esp_train_results/2020-11/carla_town01_B10_A5_T20_precog_SocialConvRNN"
overrides = [
    "dataset=carla_town01_A5_T20_test",
    "model.directory={}".format(MODEL_DIRECTORY),
    "main.compute_metrics=false"]
with initialize(config_path="precog/conf"):
    cfg = compose(config_name="esp_infer_config", overrides=overrides)
    print(OmegaConf.to_yaml(cfg))

/home/fireofearth/miniconda3/envs/precog/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fireofearth/miniconda3/envs/precog/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fireofearth/miniconda3/envs/precog/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fireofearth/miniconda3

main:
  debug_bijection: false
  compute_metrics: false
  plot: true
model:
  directory: /home/fireofearth/code/data/esp_train_results/2020-11/carla_town01_B10_A5_T20_precog_SocialConvRNN
hardware:
  allow_growth: true
  per_process_gpu_memory_fraction: 0.3
images:
  ext: jpg
  figsize:
  - 8
  - 8
split: test
dataset:
  plot_allchannel: false
  class: precog.dataset.serialized_dataset.SerializedDataset
  params:
    root_path: /home/fireofearth/code/data/precog_carla_dataset/town01
    _max_A: 5
    B: 10
    T: 20
    T_past: 10
    load_bev: true
    sdt_bev: false
    feature_pixels_per_meter: 2.0
    W: 100
    fmt: json
    val_suffix: /val/
    test_suffix: /test/
    train_suffix: /train_generated/
    match_prefix: ma_*
    _name: carla_town01_A5_T20_test
plotting:
  bev_kwargs:
    onechannel: false
    allchannel: false
    fmt: carla



/home/fireofearth/miniconda3/envs/precog/lib/python3.6/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive dataset/carla_town01_A5_T20_test.yaml in file:///home/fireofearth/code/robotics/precog/precog/conf.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)


In [2]:
sess = tfutil.create_session(
    allow_growth=cfg.hardware.allow_growth,
    per_process_gpu_memory_fraction=cfg.hardware.per_process_gpu_memory_fraction)

`tfutil.load_annotated_model()` uses `tf.compat.v1.train.latest_checkpoint()` to get the latest TF session checkpoint; uses `tf.compat.v1.train.import_meta_graph` to recreate and restore the graph to the current TF session. 

Then we read the tensor collection names from disk create `tensor_collections` object by calling `tf.compat.v1.get_collection()`  on the names.

In [3]:
ckpt, graph, tensor_collections = tfutil.load_annotated_model(cfg.model.directory, sess)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/fireofearth/code/data/esp_train_results/2020-11/carla_town01_B10_A5_T20_precog_SocialConvRNN/esp-model-15000


In [5]:
global_step = tf.compat.v1.train.get_or_create_global_step()
print(global_step)
step = sess.run(global_step)
print(step)

criterion = tf.get_default_graph().get_tensor_by_name("H_pq:0")
Hpq = tf.get_default_graph().get_tensor_by_name("H_pq:0")
ehat = tf.get_default_graph().get_tensor_by_name("ehat:0")
# model_distribution.variables
#
#


<tf.Variable 'global_step:0' shape=() dtype=int64_ref>
15000


<tf.Tensor 'H_pq:0' shape=(10, 12) dtype=float64>

In [7]:
optimizer = tf.compat.v1.train.RMSPropOptimizer(4e-4)
print(optimizer.name)

In [8]:
print(optimizer.name)

AttributeError: 'RMSPropOptimizer' object has no attribute 'name'

In [6]:
for name, coll in tensor_collections.items():
    print(name)
    o = '   ' + ' '.join(map(lambda t: t.name, coll))
    print(o)
    print()

sample_input
   S_past_world_frame:0 yaws:0 overhead_features:0 agent_presence:0 light_strings:0 is_training:0 Z_sample:0

infer_input
   S_past_world_frame:0 yaws:0 overhead_features:0 agent_presence:0 light_strings:0 S_future_world_frame:0 is_training:0

shared_input
   agent_presence:0 is_training:0 light_strings:0 yaws:0 S_past_world_frame:0 overhead_features:0

sample_output
   Z:0 mu:0 S_grid_frame:0 S_world_frame:0 m:0 sigma:0 S_car_frames:0 sigel:0 log_q_samples:0

infer_output
   log_q_expert:0

sample_metric
   mean_MHat:0 mean_Hqphat:0 mhat:0

infer_metric
   mean_Hpq:0 mean_ehat:0 H_pq:0 ehat:0

intermediate_input
   S_past_car_frames:0 S_past_grid_frame:0 agent_counts:0

intermediate_label
   S_future_car_frames:0 S_future_grid_frame:0



In [ ]:
# interface that contains and is used to run the trained model
inference = interface.ESPInference(tensor_collections)

In [ ]:
# load the dataset
dataset = hydra.utils.instantiate(cfg.dataset, **cfg.dataset.params)

In [ ]:
# load a minibatch from the dataset
minibatch = dataset.get_minibatch(
    split=cfg.split,
    input_singleton=inference.training_input,
    is_training=False)

In [ ]:
# I can use tensorboard to display the computation graph when running the network
# the graph is very messy. To read it call
# tensorboard --logdir=./log
# writer = tf.summary.FileWriter("log", sess.graph)

In [ ]:
# generate the latent variables used in the ESP network
# prepare to run the TF session with minibatch as input
sessrun = functools.partial(sess.run, feed_dict=minibatch)

In [ ]:
# generate the latent variables used in the ESP network
base_and_log_q = inference.base_and_log_q.to_numpy(sessrun)
print(base_and_log_q.Z_sample.shape)
print(base_and_log_q.Z_sample[0,0,0,0,:])

# sampled latent variables for trained networks have dimensions (10, 12, 5, 20, 2)
# and are randomly sampled at every call within the graph as evident from below
# [1.03849122   1.23841257]
# [-0.02836075 -0.10323571]
# [-0.06120932 -0.65453399]
# [1.29992859   0.30881844]
# ... etc

In [ ]:
# generate the latent variables used in the ESP network
# from the model's variables themselves, bypassing the interface
coll = tensor_collections
Z_sample = coll['sample_input'][6]
print(Z_sample)
o = sessrun(Z_sample)
print(o.shape)
print(o[0,0,0,0,:])
# for k, v in tensor_collections.items():
#     print(k)
#     print(v)

In [ ]:
# check that S_past_world_frame is loaded from past trajectory
coll = tensor_collections
S_past_world_frame = coll['sample_input'][0]
o = sessrun(S_past_world_frame)
print(o.shape)
print(o[0,0,0,:])

In [ ]:
# check that S_past_world_frame is loaded from past trajectory
# using Tensor.eval() instead
coll = tensor_collections
Z_sample = coll['sample_input'][6]
print(Z_sample)
o = Z_sample.eval(session=sess)#, feed_dict=minibatch)
print(o.shape)
print(o[0,0,0,0,:])

In [ ]:
coll = tensor_collections
S_grid_frame = coll['sample_output'][2]
o = Z_sample.eval(session=sess, feed_dict=minibatch)
print(o.shape)
print(o[0,0,0,0,:])

In [ ]:
for k, _ in minibatch.items():
    print(k)

In [ ]:
distribution = tfp.distributions.MultivariateNormalDiag(
            loc=tf.constant(0., dtype=tf.float64), scale_diag=tf.constant([1., 1.], dtype=tf.float64))
sample = distribution.sample(
    sample_shape=(
        inference.metadata.B, 
        inference.metadata.K, 
        inference.metadata.A, 
        inference.metadata.T))
sample = sess.run(sample)

In [ ]:
# finally figured out how to fix the normal sampling
coll = tensor_collections
Z_sample = coll['sample_input'][6]
Z = coll['sample_output'][0]
o1, o2 = sess.run([Z, Z_sample], feed_dict={Z_sample: sample})
print(o1.shape)
print(o1[0,0,0,0,:])
print(o2[0,0,0,0,:])